In [ ]:
from lstm2 import *
import pickle
import numpy as np
import numpy
import pickle
from random import *
import theano.tensor as T
def _p(pp, name):
    return '%s_%s' % (pp, name)
import re

<br>Xavier Initialisation </br>
<br>The Bias of the forget gates <b>b[n:n+n])</b> are set to 1.5

In [ ]:

def getlayerx(d,pref,n,nin):
    mu=0.0
    sigma=0.1
    U = np.random.normal(mu,sigma, (4*n,n))/np.sqrt(nin)
    U=np.array(U,dtype=np.float32)
    W =np.random.normal(mu,sigma, (4*n,nin))/np.sqrt(nin)
    W=np.array(W,dtype=np.float32)
    
    d[_p(pref, 'U')] = U
    #b = numpy.zeros((n * 300,))+1.5
    b = np.random.uniform(-0.5,0.5,size=(4*n,))
    b[n:n*2]=1.5 
    d[_p(pref, 'W')] = W
    d[_p(pref, 'b')] = b.astype(config.floatX)
    return d
def creatrnnx():
    newp=OrderedDict()
    print ("Creating neural network")
    newp=getlayerx(newp,'1lstm1',50,300)
  
    newp=getlayerx(newp,'2lstm1',50,300)
    
    return newp

The gensim word2vec has 300 dimension vectors
<br> <b>Note: </b> We don't train the embedding vectors anymore

In [ ]:
prefix='lstm'
dim_proj=300 
noise_std=0.
options=locals().copy()





In [ ]:
newp=creatrnnx()

Make the weights of both LSTM's equal LSTM_A and LSTM_B
<br> Prefix '1' is used to represent LSTM_A
<br> Prefix '2' for LSTM_B
<br>Suffix "_U" for candidature
<br> "_b" for Bias gates
<br> "_W" for input gate

In [ ]:
for i in newp.keys():
    if i[0]=='1':
        newp[i]=newp['2'+i[1:]]
        print '2'+i[1:]

Copy to GPU

In [ ]:
y = tensor.vector('y', dtype='float32')
mask11 = tensor.matrix('mask11', dtype=config.floatX)
mask21 = tensor.matrix('mask21', dtype=config.floatX)
emb11=theano.tensor.ftensor3('emb11')
emb21=theano.tensor.ftensor3('emb21')

#newp=pickle.load(open("33814200semevalorig.p","rb"))

tnewp=init_tparams(newp)
trng = RandomStreams(1234)
use_noise = theano.shared(numpy_floatX(0.))




LSTM Layer
<br> nhd=number of hidden units
<br>"prefix"-> Name of the LSTM (Stored in the form of a dictionary)

In [ ]:
def lstm_layer2(tparams, state_below, options, prefix='lstm', mask=None,nhd=None):
    nsteps = state_below.shape[0]
    if state_below.ndim == 3:
        n_samples = state_below.shape[1]
    else:
        n_samples = 1

    assert mask is not None

    def _slice(_x, n, dim):
        if _x.ndim == 3:
            return _x[:, :, n * dim:(n + 1) * dim]
        return _x[:, n * dim:(n + 1) * dim]

    def _step(m_, x_, h_, c_):
        preact = tensor.dot(h_, tparams[_p(prefix, 'U')].T)
        preact += x_
        preact += tparams[_p(prefix, 'b')]

        i = tensor.nnet.sigmoid(_slice(preact, 0, nhd))
        f = tensor.nnet.sigmoid(_slice(preact, 1, nhd))
        o = tensor.nnet.sigmoid(_slice(preact, 2, nhd))
        c = tensor.tanh(_slice(preact, 3, nhd))

        c = f * c_ + i * c
        c = m_[:, None] * c + (1. - m_)[:, None] * c_

        h = o * tensor.tanh(c)
        h = m_[:, None] * h + (1. - m_)[:, None] * h_

        return [h, c]

    state_below = (tensor.dot(state_below, tparams[_p(prefix, 'W')].T) +
                   tparams[_p(prefix, 'b')])
    #print "hvals"
    dim_proj = nhd
    [hvals,yvals], updates = theano.scan(_step,
                                sequences=[mask, state_below],
                                outputs_info=[tensor.alloc(numpy_floatX(0.),
                                                           n_samples,
                                                           dim_proj),
                                              tensor.alloc(numpy_floatX(0.),
                                                           n_samples,
                                                           dim_proj)],
                                name=_p(prefix, '_layers'),
                                n_steps=nsteps)
    return hvals

In [ ]:
def dropout_layer(state_before, use_noise, rrng,rate):
    proj = tensor.switch(use_noise,
                         (state_before *rrng),
                         state_before * (1-rate))
    return proj

Simplified function to add a layer
<br>To enable Dropout (optional) set used=<font color="green">True

In [ ]:
def getpl(prevlayer,pre,mymask,used,rrng,size):
    proj = lstm_layer(tnewp, prevlayer, options,
                                        prefix=pre,
                                        mask=mymask,nhd=size)
    #proj = (proj * mymask[:, :, None]).sum(axis=0)
    #proj = proj / mymask.sum(axis=0)[:, None]
    if used:
        print "Added dropout"
        proj = dropout_layer(proj, use_noise, rrng,0.5)
        
    return proj

rrng->Used when Dropout is enabled
<br> <b>proj11</b> takes the inputs as embedding matrix <b>emb1</b> and gives the o/p of the LSTM_A

In [ ]:

rate=0.5
rrng=trng.binomial((mask11.shape[0],mask11.shape[1],50),p=1-rate, n=1,dtype=emb11.dtype)

proj11=getpl2(emb11,'1lstm1',mask11,False,rrng,50)[-1]


proj21=getpl2(emb21,'2lstm1',mask21,False,rrng,50)[-1]


Similarity= exp(-L1_norm(difference of outputs))
$$sim = e^{(-|proj11-proj21|)}$$

Since the input is in the format:
<br>
(Max no. of words in batch, No. of Samples, 300)
<br> The output proj11[-1] will have dimensions( No. of samples, Hidden unit dimension), so (No. of samples,50)

In [ ]:
# norm similarity  symmetric

batchsize=64
bts=float(batchsize)

dif=(proj21-proj11).norm(L=1,axis=1)


#dropout diff
#dif=(proj11-proj21).norm(L=1,axis=0)


#si=T.diagonal(T.dot(proj12,proj22.T))/T.nnet.softmax(dif)
#sim=T.reshape(si,(si.shape[1],))
s2=T.exp(-dif)
sim=T.clip(s2,1e-7,1.0-1e-7)
lr = tensor.scalar(name='lr')
ys=T.clip((y-1.0)/4.0,1e-7,1.0-1e-7)
cost=T.mean((sim - ys) ** 2)
ns=emb11.shape[1]
f2sim=theano.function([emb11,mask11,emb21,mask21],sim,allow_input_downcast=True)
f_cost=theano.function([emb11,mask11,emb21,mask21,y],cost,allow_input_downcast=True)






The gradi refers to gradients wrt. cost, and is a list containing gradients to be update weights
<br> We average out the gradients by appending to another list grads[]
<br> So, we average out the gradients : wrt LSTM_A and wrt LSTM_B
<br> i.e, gradient= (grad(wrt(LSTM_A)+grad(wrt(LSTM_B))/2.0 to maintain the symmetricity between either LSTMs

In [ ]:
gradi = tensor.grad(cost, wrt=tnewp.values())#/bts
#newp[i]=newp['2'+i[1:]]
grads=[]
l=len(gradi)
for i in range(0,l/2):
    gravg=(gradi[i]+gradi[i+l/2])/(2.0)
    #print i,i+9
    grads.append(gravg)
for i in range(0,len(tnewp.keys())/2):
    grads.append(grads[i])

Optimisers
<br> Recommended: Adadelta

In [ ]:
def adadelta(lr, tparams, grads, emb11,mask11,emb21,mask21,y, cost):
    zipped_grads = [theano.shared(p.get_value() * numpy_floatX(0.),
                                  name='%s_grad' % k)
                    for k, p in tparams.iteritems()]
    running_up2 = [theano.shared(p.get_value() * numpy_floatX(0.),
                                 name='%s_rup2' % k)
                   for k, p in tparams.iteritems()]
    running_grads2 = [theano.shared(p.get_value() * numpy_floatX(0.),
                                    name='%s_rgrad2' % k)
                      for k, p in tparams.iteritems()]

    zgup = [(zg, g) for zg, g in zip(zipped_grads, grads)]
    rg2up = [(rg2, (0.95 * rg2 + 0.05* (g ** 2)))
             for rg2, g in zip(running_grads2, grads)]

    f_grad_shared = theano.function([emb11,mask11,emb21,mask21,y], cost, updates=zgup + rg2up,allow_input_downcast=True,
                                    name='adadelta_f_grad_shared')

    updir = [-tensor.sqrt(ru2 + 1e-6) / tensor.sqrt(rg2 + 1e-6) * zg
             for zg, ru2, rg2 in zip(zipped_grads,
                                     running_up2,
                                     running_grads2)]
    ru2up = [(ru2, (0.95 * ru2 + 0.05 * (ud ** 2)))
             for ru2, ud in zip(running_up2,updir)]
    param_up = [(p, p + ud) for p, ud in zip(tparams.values(), updir)]

    f_update = theano.function([lr], [], updates=ru2up + param_up,
                               on_unused_input='ignore',
                               name='adadelta_f_update')

    return f_grad_shared, f_update

def sgd(lr, tparams, grads, emb11,mask11,emb21,mask21,y, cost):
    
    gshared = [theano.shared(p.get_value() * 0., name='%s_grad' % k)
               for k, p in tparams.iteritems()]
    gsup = [(gs, g) for gs, g in zip(gshared, grads)]
    f_grad_shared = theano.function([emb11,mask11,emb21,mask21,y], cost, updates=gsup,
                                    name='sgd_f_grad_shared')
    pup = [(p, p - lr * g) for p, g in zip(tparams.values(), gshared)]
    f_update = theano.function([lr], [], updates=pup,
                               name='sgd_f_update')

    return f_grad_shared, f_update


def rmsprop(lr, tparams, grads, emb11,mask11,emb21,mask21,y, cost):
    zipped_grads = [theano.shared(p.get_value() * numpy_floatX(0.),
                                  name='%s_grad' % k)
                    for k, p in tparams.iteritems()]
    running_grads = [theano.shared(p.get_value() * numpy_floatX(0.),
                                   name='%s_rgrad' % k)
                     for k, p in tparams.iteritems()]
    running_grads2 = [theano.shared(p.get_value() * numpy_floatX(0.),
                                    name='%s_rgrad2' % k)
                      for k, p in tparams.iteritems()]

    zgup = [(zg, g) for zg, g in zip(zipped_grads, grads)]
    rgup = [(rg, 0.95 * rg + 0.05 * g) for rg, g in zip(running_grads, grads)]
    rg2up = [(rg2, 0.95 * rg2 + 0.05 * (g ** 2))
             for rg2, g in zip(running_grads2, grads)]

    f_grad_shared = theano.function([emb11,mask11,emb21,mask21,y], cost,
                                    updates=zgup + rgup + rg2up,
                                    name='rmsprop_f_grad_shared')

    updir = [theano.shared(p.get_value() * numpy_floatX(0.),
                           name='%s_updir' % k)
             for k, p in tparams.iteritems()]
    updir_new = [(ud, 0.9 * ud - 1e-4 * zg / tensor.sqrt(rg2 - rg ** 2 + 1e-4))
                 for ud, zg, rg, rg2 in zip(updir, zipped_grads, running_grads,
                                            running_grads2)]
    param_up = [(p, p + udn[1])
                for p, udn in zip(tparams.values(), updir_new)]
    f_update = theano.function([lr], [], updates=updir_new + param_up,
                               on_unused_input='ignore',
                               name='rmsprop_f_update')

    return f_grad_shared, f_update

In [ ]:
#Takes a long time to compute
f_grad_shared, f_update = adadelta(lr, tnewp, grads,
                                    emb11,mask11,emb21,mask21,y, cost)
